# Aggregate 
Following on from "rt_daily_aggregations" (which, for RTD, has been slimmed down to processing data), aggregates observations

In [1]:
import pandas as pd
import dask.dataframe as dd
import boto3

In [3]:
""" Assemble pb to parquet dataframes, one per day """
import sys
from google.protobuf.json_format import MessageToJson
import json
from google.transit import gtfs_realtime_pb2
from io import BytesIO
import boto3
import pandas as pd
import pendulum
import dask.dataframe as dd

In [4]:
# Step 

## Step 1
Create a dask dataframe from parquet files.
For development, add method to GTFSdata . Update combine_daily_files later

In [22]:
import sys
import os
os.getcwd()
from pathlib import Path
str(Path.cwd().parent.parent)
# sys.path.append
# sys.path
sys.path.append(str(Path.cwd().parent.parent))
# from  scrape_data.parse_data import gtfs_decode
from scrape_data.combine_daily_files import GTFSdata, VehiclePosition, TripUpdate, Alerts

In [8]:
def generate_dask_dataframe(self):
    s3_prefix = f"processed/{self.data_type}/*.parquet"
    

In [23]:
s3_prefix = f"processed/TripUpdate/*.parquet"
s3_path = f"s3://rtd-ghost-buses-private/{s3_prefix}"
s3_path

's3://rtd-ghost-buses-private/processed/TripUpdate/*.parquet'

In [31]:
dask_df = dd.read_parquet(s3_path)
dask_df.head()

,id,tripUpdate.trip.tripId,tripUpdate.trip.scheduleRelationship,tripUpdate.trip.routeId,tripUpdate.trip.directionId,tripUpdate.stopTimeUpdate,tripUpdate.vehicle.id,tripUpdate.vehicle.label,header.timestamp,filename
0,1672556583_114253564,114253564,CANCELED,101E,0,None,None,None,2023-01-01 07:03:03,bus_data_TripUpdate/2023-01-01T00:03:12.767143...
0,1672556583_114253883,114253883,CANCELED,101E,1,None,None,None,2023-01-01 07:03:03,bus_data_TripUpdate/2023-01-01T00:03:12.767143...
0,1672556583_114253884,114253884,CANCELED,101E,1,None,None,None,2023-01-01 07:03:03,bus_data_TripUpdate/2023-01-01T00:03:12.767143...
0,1672556583_114254175,114254175,CANCELED,101H,0,None,None,None,2023-01-01 07:03:03,bus_data_TripUpdate/2023-01-01T00:03:12.767143...
0,1672556583_114254183,114254183,CANCELED,101H,0,None,None,None,2023-01-01 07:03:03,bus_data_TripUpdate/2023-01-01T00:03:12.767143...


In [50]:
s3_onefile_path = f"s3://rtd-ghost-buses-private/processed/TripUpdate/2023-01-09.parquet"
parquet_df = pd.read_parquet(s3_onefile_path)



In [77]:
# Vehicle position
s3_position_onefile_path = f"s3://rtd-ghost-buses-private/processed/VehiclePosition/2023-01-09.parquet"
parquet_position_df = pd.read_parquet(s3_position_onefile_path)


In [80]:
parquet_position_df # ['vehicle.currentStatus'].value_counts() # in transit to/stopped at/None

,id,vehicle.position.latitude,vehicle.position.longitude,vehicle.position.bearing,vehicle.timestamp,vehicle.vehicle.id,vehicle.vehicle.label,vehicle.trip.tripId,vehicle.trip.scheduleRelationship,vehicle.trip.routeId,vehicle.trip.directionId,vehicle.currentStatus,vehicle.stopId,header.timestamp,filename
0,1673247791_1520,39.740130,-104.836334,91.0,1673247758,1520,1520,None,None,None,NaN,None,None,2023-01-09 07:03:11,bus_data_VehiclePosition/2023-01-09T00:03:13.1...
0,1673247791_1523,39.738483,-104.823320,273.0,1673247685,1523,1523,None,None,None,NaN,None,None,2023-01-09 07:03:11,bus_data_VehiclePosition/2023-01-09T00:03:13.1...
0,1673247791_1552,39.770350,-104.984720,0.0,1673247751,1552,1552,None,None,None,NaN,None,None,2023-01-09 07:03:11,bus_data_VehiclePosition/2023-01-09T00:03:13.1...
0,1673247791_1662,39.742170,-104.987970,129.0,1673247764,1662,1662,None,None,None,NaN,None,None,2023-01-09 07:03:11,bus_data_VehiclePosition/2023-01-09T00:03:13.1...
0,1673247791_1992,40.016933,-105.249150,0.0,1673247743,1992,1992,None,None,None,NaN,None,None,2023-01-09 07:03:11,bus_data_VehiclePosition/2023-01-09T00:03:13.1...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,1673333886_F1E2094DF58CD483E053DD4D1FAC9DB5,39.772346,-104.905370,180.0,1673333838,F1E2094DF58CD483E053DD4D1FAC9DB5,4,114363332,SCHEDULED,A,1.0,None,34468,2023-01-10 06:58:06,bus_data_VehiclePosition/2023-01-09T23:58:13.2...
0,1673333886_F1E2094DF596D483E053DD4D1FAC9DB5,39.792350,-105.112650,0.0,1673333821,F1E2094DF596D483E053DD4D1FAC9DB5,32,114351623,SCHEDULED,113G,1.0,None,34510,2023-01-10 06:58:06,bus_data_VehiclePosition/2023-01-09T23:58:13.2...
0,1673333886_F1E2094DF5A3D483E053DD4D1FAC9DB5,39.797430,-105.086660,180.0,1673333794,F1E2094DF5A3D483E053DD4D1FAC9DB5,31,114351526,SCHEDULED,113G,0.0,None,34525,2023-01-10 06:58:06,bus_data_VehiclePosition/2023-01-09T23:58:13.2...
0,1673333886_F1E2094DF5A8D483E053DD4D1FAC9DB5,39.807404,-104.784004,0.0,1673333705,F1E2094DF5A8D483E053DD4D1FAC9DB5,2,114363246,SCHEDULED,A,0.0,None,34475,2023-01-10 06:58:06,bus_data_VehiclePosition/2023-01-09T23:58:13.2...


In [53]:
parquet_df['tripUpdate.vehicle.id'].isna().value_counts() #.columns
parquet_df.loc[~parquet_df['tripUpdate.vehicle.id'].isna()]
parquet_df.loc[parquet_df['id'] == '1673247789_114354115', 'tripUpdate.stopTimeUpdate'][0] 
# What does the stop time update actually look like? Arrival + departure time for each stopId/stopSequence
parquet_df.loc[parquet_df['id'] == '1673247789_114354115']
# tripUpdate.trip.tripId	tripUpdate.trip.scheduleRelationship	tripUpdate.trip.routeId	tripUpdate.trip.directionId
# 114354115	SCHEDULED	15	0



,id,tripUpdate.trip.tripId,tripUpdate.trip.scheduleRelationship,tripUpdate.trip.routeId,tripUpdate.trip.directionId,tripUpdate.stopTimeUpdate,tripUpdate.vehicle.id,tripUpdate.vehicle.label,header.timestamp,filename
0,1673247789_114354115,114354115,SCHEDULED,15,0,"[{'arrival': {'time': '1673247831'}, 'departur...",6394,6394,2023-01-09 07:03:09,bus_data_TripUpdate/2023-01-09T00:03:12.687686...


In [76]:
# look at schedule? Convert 
parquet_df.loc[parquet_df['id'] == '1673247789_114354115', 'tripUpdate.stopTimeUpdate'][0] 
stop_time_df = pd.DataFrame.from_records(parquet_df.loc[parquet_df['id'] == '1673247789_114354115', 'tripUpdate.stopTimeUpdate'][0] )
stop_time_df['arrival'] = stop_time_df['arrival'].apply(pd.Series)
stop_time_df['departure'] = stop_time_df['departure'].apply(pd.Series)
stop_time_df['arrival'] = pd.to_datetime(stop_time_df['arrival'], unit='s',utc=True).dt.tz_convert('US/Mountain')
stop_time_df['departure'] = pd.to_datetime(stop_time_df['departure'], unit='s',utc=True).dt.tz_convert('US/Mountain')
stop_time_df
# Note that schedule is tz-naive, but stop time updates are Posix seconds since 1970 UTC

/var/folders/1t/6fjth5nx10z8_4tgz392m7vc0000gn/T/ipykernel_14707/1425818439.py:4: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  stop_time_df['arrival'] = stop_time_df['arrival'].apply(pd.Series)
/var/folders/1t/6fjth5nx10z8_4tgz392m7vc0000gn/T/ipykernel_14707/1425818439.py:5: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  stop_time_df['departure'] = stop_time_df['departure'].apply(pd.Series)
/var/folders/1t/6fjth5nx10z8_4tgz392m7vc0000gn/T/ipykernel_14707/1425818439.py:5: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  stop_time_df['departure'] = stop_time_df['departure'].apply(pd.Series)


,arrival,departure,scheduleRelationship,stopId,stopSequence
0,2023-01-09 00:03:51-07:00,2023-01-09 00:03:51-07:00,SCHEDULED,20745,2
1,2023-01-09 00:04:34-07:00,2023-01-09 00:06:00-07:00,SCHEDULED,17717,3
2,2023-01-09 00:08:00-07:00,2023-01-09 00:08:00-07:00,SCHEDULED,17714,4
3,2023-01-09 00:09:28-07:00,2023-01-09 00:09:28-07:00,SCHEDULED,17711,5
4,2023-01-09 00:11:22-07:00,2023-01-09 00:11:22-07:00,SCHEDULED,17730,6
5,2023-01-09 00:13:00-07:00,2023-01-09 00:13:00-07:00,SCHEDULED,12849,7
6,2023-01-09 00:13:44-07:00,2023-01-09 00:13:44-07:00,SCHEDULED,12978,8
7,2023-01-09 00:15:10-07:00,2023-01-09 00:15:10-07:00,SCHEDULED,12953,9
8,2023-01-09 00:15:53-07:00,2023-01-09 00:15:53-07:00,SCHEDULED,12861,10
9,2023-01-09 00:17:00-07:00,2023-01-09 00:17:00-07:00,SCHEDULED,12870,11


In [98]:
# parquet_df.loc[parquet_df['tripUpdate.trip.scheduleRelationship']== 'CANCELED']
parquet_df['tripUpdate.trip.scheduleRelationship'].value_counts()

SCHEDULED    97329
CANCELED      3245
Name: tripUpdate.trip.scheduleRelationship, dtype: int64

In [72]:
parquet_df.loc[parquet_df['id'] == '1673247789_114354115'].iloc[0]

id                                                                   1673247789_114354115
tripUpdate.trip.tripId                                                          114354115
tripUpdate.trip.scheduleRelationship                                            SCHEDULED
tripUpdate.trip.routeId                                                                15
tripUpdate.trip.directionId                                                             0
tripUpdate.stopTimeUpdate               [{'arrival': {'time': '1673247831'}, 'departur...
tripUpdate.vehicle.id                                                                6394
tripUpdate.vehicle.label                                                             6394
header.timestamp                                                      2023-01-09 07:03:09
filename                                bus_data_TripUpdate/2023-01-09T00:03:12.687686...
Name: 0, dtype: object

In [32]:
dask_df.columns

Index(['id', 'tripUpdate.trip.tripId', 'tripUpdate.trip.scheduleRelationship',
       'tripUpdate.trip.routeId', 'tripUpdate.trip.directionId',
       'tripUpdate.stopTimeUpdate', 'tripUpdate.vehicle.id',
       'tripUpdate.vehicle.label', 'header.timestamp', 'filename'],
      dtype='object')

In [27]:
dask_df['tripUpdate.stopTimeUpdate'].isna().value_counts().compute()
# tripUpdate.vehicle.id
# 	tripUpdate.vehicle.label	header.timestam


False    8776127
True      398444
Name: tripUpdate.stopTimeUpdate, dtype: int64

In [39]:
dask_df["tripUpdate.vehicle.id"].isna().value_counts().compute(engine="fastparquet")

KeyError: "None of [Index(['tripUpdate.vehicle.id'], dtype='object')] are in the [columns]"

In [40]:
dask_df['tripUpdate.vehicle.label'].isna().value_counts().compute()


KeyError: "None of [Index(['tripUpdate.vehicle.label'], dtype='object')] are in the [columns]"

In [25]:
len(dask_df)

9174571

In [ ]:
# TODO - Aggregate! How?
# By date+hour
# 'rt' is route, maps to ???
# 'des' is destination, maps to ???

# Is chicago data 

# Old code: 
# ["tripUpdate.trip.tripId"]
# data['data_time'] = pd.to_datetime(data['tmstmp'], format='%Y%m%d %H:%M')

# data['data_hour'] = data.data_time.dt.hour
# data['data_date'] = data.data_time.dt.date

# bucket.put_object(Body = data.to_csv(index = False), 
#             Key = f'bus_full_day_data_v2/{date_str}.csv')

# # combine vids into a set (drops duplicates): https://stackoverflow.com/a/45925961
# hourly_summary = data.groupby(['data_date', 'data_hour', 'rt', 'des']).agg({'vid': set, 'tatripid': set, 'tablockid': set}).reset_index()
# # get number of vehicles per hour per route
# hourly_summary['vh_count'] = hourly_summary['vid'].apply(len)
# hourly_summary['trip_count'] = hourly_summary['tatripid'].apply(len)
# hourly_summary['block_count'] = hourly_summary['tablockid'].apply(len)

In [9]:
GTFSdata.generate_dask_dataframe = generate_dask_dataframe

In [81]:
s3_alerts_onefile_path = f"s3://rtd-ghost-buses-private/processed/Alerts/2023-01-09.parquet"
parquet_alerts_df = pd.read_parquet(s3_alerts_onefile_path)


In [91]:
parquet_alerts_df['alert.headerText.translation'].iloc[1]
parquet_alerts_df.iloc[6]

id                                                                               26611
alert.activePeriod                      [{'end': '1693558740', 'start': '1666801120'}]
alert.informedEntity                 [{'agencyId': 'RTD', 'routeId': '116X', 'route...
alert.cause                                                                OTHER_CAUSE
alert.effect                                                                    DETOUR
alert.headerText.translation         [{'language': 'en', 'text': 'Route 116X detour...
alert.descriptionText.translation    [{'language': 'en', 'text': 'Affected stops:
...
header.timestamp                                                   2023-01-09 07:01:54
filename                             bus_data_Alerts/2023-01-09T00:03:12.015144-07:...
Name: 0, dtype: object

In [97]:
parquet_alerts_df[['alert.effect', 'alert.cause']].value_counts().sort_index()

alert.effect      alert.cause      
DETOUR            CONSTRUCTION          864
                  OTHER_CAUSE           360
                  TECHNICAL_PROBLEM     174
MODIFIED_SERVICE  UNKNOWN_CAUSE        7200
NO_SERVICE        CONSTRUCTION         8837
                  MEDICAL_EMERGENCY      39
                  OTHER_CAUSE          1207
                  TECHNICAL_PROBLEM      77
                  UNKNOWN_CAUSE         309
OTHER_EFFECT      MAINTENANCE            75
                  TECHNICAL_PROBLEM      41
UNKNOWN_EFFECT    UNKNOWN_CAUSE         288
dtype: int64

In [93]:
parquet_alerts_df['alert.cause'].value_counts()

NO_SERVICE          10469
MODIFIED_SERVICE     7200
DETOUR               1398
UNKNOWN_EFFECT        288
OTHER_EFFECT          116
Name: alert.effect, dtype: int64